In [3]:
import json
import tensorflow as tf
import pandas as pd

In [6]:
text_a=[]
text_b=[]
text_c=[]
label=[]
guid=[]
data_path ="../input/SCM_5k.json"
import re
def remove_punctuation(line):
    rule = re.compile(r"[^\u4e00-\u9fa5]")
    line = rule.sub('',line)
    return line

with open(data_path, encoding='utf-8') as reader:
    for i,line in enumerate(reader):
        lines = json.loads(line)

        a =  lines["A"].split("\n\n")[1]
        b =  lines['B'].split("\n\n")[1]
        c =  lines['C'].split("\n\n")[1]
        text_a.append(a)
        text_b.append(b)
        text_c.append(c)


In [13]:
aaa=pd.DataFrame([text_a,text_b,text_c]).T
aaa.columns=['A','B','C']
text_c[0]

'原告牛某1向本院提出诉讼请求：1．请求法院判令三被告立即共同连带归还原告借款本金3000000元，并按照年利率24％的标准支付原告自2013年6月10日起至三被告实际还清全部欠款之日的利息，该利息暂计至2017年11月9日为3230000元；2．请求法院判令三被告承担本案全部诉讼费用。事实和理由：2011年11月2日，原告与被告冯某1、被告康某1签订了《借款协议书》，约定原告出借给被告冯某1、被告康某1人民币300万元，借款期限为12个月，自2011年11月2日至2012年10月31日。双方约定按每月3％计算利息，被告按季度向原告支付利息。上述合同签订后，原告依约向被告支付了全部款项，但被告一直未能按时支付利息，借款期限届满后也未能归还本金。2014年2月10日，被告山西智伟基业房地产开发有限公司向原告出具《承诺书》，明确了其向原告借款的事实，并承诺于2014年3月、6月向原告支付利息，于2014年11月2日前向原告还清全部本息。该承诺书出具后，原告与被告冯某1、被告康某1于2014年3月5日签订了《借款补充协议书》，约定将前述借款延期至2014年11月2日。但借款期限届满后三被告仍未依约还款，经原告多次催要无果，故诉至法院，请求法院依法支持原告的诉讼请求。被告山西智伟基业房地产开发有限公司、冯某1、康某1承认原告牛某1提出的全部诉讼请求。\n'

In [3]:
from keras_bert import get_base_dict
token_dict=get_base_dict()
with open("./chinese_L-12_H-768_A-12/vocab.txt") as f:
    for token in f:
        token = token.strip()
        if token not in token_dict:
            token_dict[token] = len(token_dict)




Using TensorFlow backend.


In [ ]:
token_dict

In [4]:
import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
tokenizer = Tokenizer(token_dict)


def load_data(text_a,text_b,label,SEQ_LEN):
    global tokenizer
    indices_a, indices_b, sentiments = [], [],[]
    for line_a, line_b,line_l in zip(text_a,text_b,label):
        ids_a, segments_a = tokenizer.encode(line_a[-300:], max_len=SEQ_LEN)
        ids_b, segments_b = tokenizer.encode(line_b[-300:], max_len=SEQ_LEN)
        indices_a.append(ids_a)
        indices_b.append(ids_b)        
        sentiments.append(line_l)
    items = list(zip(indices_a, indices_b, sentiments))
    np.random.shuffle(items)
    indices_a, indices_b, sentiments = zip(*items)  
    indices_a = np.array(indices_a)
    indices_b = np.array(indices_b)     
    return [indices_a, np.zeros_like(indices_a), indices_b, np.zeros_like(indices_b)], np.array(sentiments)    
    

In [5]:
train_feat,train_label = load_data(text_a,text_b,label,300)

In [12]:
str_num={"B":1,"C":0}
num_str={1:"B",0:"C"}

In [13]:
train_label = np.array(list(map(lambda x:str_num[x],train_label)))

In [14]:
import os
from keras_bert import load_trained_model_from_checkpoint
from keras.layers import GlobalAveragePooling1D
from keras.optimizers import *
from keras.layers import *
from keras.layers import Layer
from keras_bert.backend import backend as K
from keras.models import Model

class MaskedGlobalAveragePool1D(Layer):

    def __init__(self, **kwargs):
        super(MaskedGlobalAveragePool1D, self).__init__(**kwargs)
        self.supports_masking = True

    def compute_mask(self, inputs, mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return input_shape[:-2] + (input_shape[-1],)

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            inputs -= K.expand_dims((1.0 - mask) * 1e6, axis=-1)
        return K.mean(inputs, axis=-2)


def BERT_Finetune():
    layer_num = 12
    checkpoint_path = './chinese_L-12_H-768_A-12/'

    config_path = os.path.join(checkpoint_path, 'bert_config.json')
    model_path = os.path.join(checkpoint_path, 'bert_model.ckpt')
#     bert_model = load_trained_model_from_checkpoint(
#         config_path,
#         model_path,
#         training=False,
#         use_adapter=True,
#         trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(layer_num)] +
#         ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
#     )
    bert_model = load_trained_model_from_checkpoint(config_path, model_path)

    for l in bert_model.layers:
        l.trainable = True    
    
    xa1_in = Input(shape=(None,))  
    xa2_in = Input(shape=(None,))  
    xb1_in = Input(shape=(None,))  
    xb2_in = Input(shape=(None,))  
    
    xa = bert_model([xa1_in, xa2_in])
    xb = bert_model([xb1_in, xb2_in])
    xa = MaskedGlobalAveragePool1D()(xa)
    xb = MaskedGlobalAveragePool1D()(xb)
    merged = concatenate([xa, xb])
#     char_diff  = Lambda(lambda x: K.abs(x[0] - x[1]))([xa, xb]) 
#     char_angle = Lambda(lambda x: x[0] * x[1])([xa, xb])
    
#     merged = concatenate([char_diff, char_angle])
#     merged = Dropout(0.5)(merged)
#     merged = BatchNormalization(trainable=False)(merged)
#     merged = Dense(500, activation="relu",trainable=False)(merged)
    
#     merged = Dropout(0.5)(merged)
#     merged = BatchNormalization(trainable=False)(merged)
    out    = Dense(1, activation="sigmoid",trainable=False)(merged)
    
    model = Model(inputs=[ xa1_in, xa2_in, xb1_in, xb2_in], outputs=out)
    model.compile(loss='binary_crossentropy', optimizer=Nadam(lr=0.001),metrics=['accuracy'])
    return model


In [15]:
model = BERT_Finetune()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [16]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
model_2 (M

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='acc', patience=6)
plateau = ReduceLROnPlateau(monitor="acc", verbose=1, mode='max', factor=0.5, patience=3)
checkpoint = ModelCheckpoint('model.hdf5', monitor='acc', verbose=2, save_best_only=True, mode='max',save_weights_only=True)


# gpu_config = tf.ConfigProto()
# gpu_config.gpu_options.allow_growth = True
# with tf.Session(config=gpu_config) as sess:
model.fit(train_feat,train_label,
          batch_size=10,
          epochs=15,
          callbacks=[early_stopping, plateau, checkpoint])

Epoch 1/15
10204/10204 [==============================] - 1048s 103ms/step - loss: 8.0664 - acc: 0.4980
Epoch 2/15
 2060/10204 [=====>........................] - ETA: 13:36 - loss: 8.1604 - acc: 0.4922